<a href="https://colab.research.google.com/github/marina554/accounting-practice/blob/main/AutoLedger(%E8%87%AA%E5%8B%95%E4%BB%95%E5%88%86%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# =========================
# 1. Auto Journal Entry Rules
# =========================
mapping_rules = [
    {
        "condition": {
            "category": "Transportation",
            "payment_method": "Cash",
            "keywords": ["train", "bus", "taxi"]
        },
        "journal": {"debit": "Travel Expenses", "credit": "Cash"}
    },
    {
        "condition": {
            "category": "Equipment Purchase",
            "payment_method": "Bank Transfer",
            "keywords": ["PC", "desk", "chair"]
        },
        "journal": {"debit": "Equipment", "credit": "Bank Deposit"}
    },
    {
        "condition": {"category": "Sales", "payment_method": "Credit"},
        "journal": {"debit": "Accounts Receivable", "credit": "Sales"}
    }
]

# =========================
# 2. Auto Journal Function
# =========================
def auto_journal(category, payment, description, date, amount):
    for rule in mapping_rules:
        cond = rule["condition"]
        if cond["category"] == category and cond["payment_method"] == payment:
            keywords = cond.get("keywords", [])
            if not keywords or any(k in description for k in keywords):
                entry = rule["journal"]
                return {
                    "date": date,
                    "debit_account": entry["debit"],
                    "credit_account": entry["credit"],
                    "description": description,
                    "amount": amount
                }
    # No match
    return {
        "date": date,
        "debit_account": "Unassigned",
        "credit_account": "Unassigned",
        "description": description,
        "amount": amount
    }

# =========================
# 3. Batch Processing for DataFrame
# =========================
def process_dataframe(df):
    results = []
    for _, row in df.iterrows():
        result = auto_journal(
            row["category"], row["payment_method"], row["description"],
            row["date"], row["amount"]
        )
        results.append(result)
    return pd.DataFrame(results)

# =========================
# 4. Account Type Mapping for Balance Sheet
# =========================
account_type_map = {
    # Assets
    "Cash": "Assets",
    "Bank Deposit": "Assets",
    "Accounts Receivable": "Assets",
    "Advances": "Assets",
    "Suspense Payments": "Assets",
    "Prepaid Expenses": "Assets",
    "Equipment": "Assets",
    # Liabilities
    "Accounts Payable": "Liabilities",
    "Accrued Expenses": "Liabilities",
    "Accrued Liabilities": "Liabilities",
    "Deposits Received": "Liabilities",
    "Loans Payable": "Liabilities",
    # Equity
    "Capital Stock": "Equity",
    "Retained Earnings": "Equity"
}

def create_balance_sheet(df):
    debit_df = df[["debit_account", "amount"]].rename(columns={"debit_account": "account"})
    credit_df = df[["credit_account", "amount"]].rename(columns={"credit_account": "account"})
    credit_df["amount"] = credit_df["amount"] * -1
    combined = pd.concat([debit_df, credit_df])
    grouped = combined.groupby("account")["amount"].sum().reset_index()
    grouped["type"] = grouped["account"].map(account_type_map).fillna("Other")
    bs = grouped.groupby("type")["amount"].sum().reset_index()
    return grouped, bs

# =========================
# 5. Account Mapping for Profit & Loss
# =========================
pl_account_map = {
    # Revenue
    "Sales": "Revenue",
    "Interest Income": "Revenue",
    "Misc Income": "Revenue",
    # Expenses
    "Purchases": "Expense",
    "Travel Expenses": "Expense",
    "Communication Expenses": "Expense",
    "Supplies Expense": "Expense",
    "Utilities": "Expense",
    "Entertainment Expense": "Expense",
    "Interest Expense": "Expense",
    "Taxes and Dues": "Expense",
    "Depreciation": "Expense",
    "Misc Expense": "Expense"
}

def create_profit_loss(df):
    debit_df = df[["debit_account", "amount"]].rename(columns={"debit_account": "account"})
    debit_df["amount"] = debit_df["amount"] * -1  # Expense
    credit_df = df[["credit_account", "amount"]].rename(columns={"credit_account": "account"})
    combined = pd.concat([debit_df, credit_df])
    combined["type"] = combined["account"].map(pl_account_map).fillna("Other")
    grouped = combined.groupby("type")["amount"].sum().reset_index()
    grouped["amount"] = grouped["amount"].round(0)
    profit = grouped.loc[grouped["type"] == "Revenue", "amount"].sum() + \
             grouped.loc[grouped["type"] == "Expense", "amount"].sum()
    profit_row = pd.DataFrame([{"type": "Net Income", "amount": profit}])
    grouped = pd.concat([grouped, profit_row], ignore_index=True)
    return grouped

# =========================
# 6. Excel Export
# =========================
def export_to_excel(result_df, bs_table, pl_table, file_name="Accounting_Report.xlsx"):
    with pd.ExcelWriter(file_name, engine="openpyxl") as writer:
        result_df.to_excel(writer, index=False, sheet_name="Journal Entries")
        bs_table.to_excel(writer, index=False, sheet_name="Balance Sheet")
        pl_table.to_excel(writer, index=False, sheet_name="Profit and Loss")
    print(f"✅ Excel export completed: {file_name}")

# =========================
# 7. Sample Data Execution
# =========================
data = {
    "date": ["2025-10-01", "2025-10-02", "2025-10-03"],
    "category": ["Transportation", "Equipment Purchase", "Sales"],
    "payment_method": ["Cash", "Bank Transfer", "Credit"],
    "description": ["Train fee (business trip)", "Desk purchase", "Client A"],
    "amount": [500, 10000, 50000]
}

df = pd.DataFrame(data)
result_df = process_dataframe(df)
grouped_accounts, bs_table = create_balance_sheet(result_df)
pl_table = create_profit_loss(result_df)
export_to_excel(result_df, bs_table, pl_table, "Accounting_Report.xlsx")


✅ Excel export completed: Accounting_Report.xlsx


In [ ]:
import pandas as pd

# =========================
# 1. 仕訳自動判定ルール
# =========================
mapping_rules = [
    {
        "条件": {
            "区分": "交通費",
            "支払方法": "現金",
            "キーワード": ["電車", "バス", "タクシー"]
        },
        "仕訳": {"借方": "旅費交通費", "貸方": "現金"}
    },
    {
        "条件": {
            "区分": "備品購入",
            "支払方法": "銀行振込",
            "キーワード": ["パソコン", "デスク", "椅子"]
        },
        "仕訳": {"借方": "備品", "貸方": "普通預金"}
    },
    {
        "条件": {"区分": "売上", "支払方法": "掛け"},
        "仕訳": {"借方": "売掛金", "貸方": "売上高"}
    }
]

# =========================
# 2. 自動仕訳関数
# =========================
def auto_journal(category, payment, description, date, amount):
    for rule in mapping_rules:
        cond = rule["条件"]
        if cond["区分"] == category and cond["支払方法"] == payment:
            keywords = cond.get("キーワード", [])
            if not keywords or any(k in description for k in keywords):
                entry = rule["仕訳"]
                return {
                    "日付": date,
                    "借方科目": entry["借方"],
                    "貸方科目": entry["貸方"],
                    "摘要": description,
                    "金額": amount
                }
    # 該当なし
    return {
        "日付": date,
        "借方科目": "未設定",
        "貸方科目": "未設定",
        "摘要": description,
        "金額": amount
    }

# =========================
# 3. DataFrame一括処理
# =========================
def process_dataframe(df):
    results = []
    for _, row in df.iterrows():
        result = auto_journal(row["区分"], row["支払方法"], row["摘要"], row["日付"], row["金額"])
        results.append(result)
    return pd.DataFrame(results)

# =========================
# 4. B/S用勘定科目マッピング
# =========================
account_type_map = {
    # 資産
    "現金": "資産",
    "普通預金": "資産",
    "売掛金": "資産",
    "立替金": "資産",
    "仮払金": "資産",
    "前払費用": "資産",
    "備品": "資産",
    # 負債
    "買掛金": "負債",
    "未払金": "負債",
    "未払費用": "負債",
    "預り金": "負債",
    "借入金": "負債",
    # 純資産
    "資本金": "純資産",
    "繰越利益剰余金": "純資産"
}

def create_balance_sheet(df):
    debit_df = df[["借方科目", "金額"]].rename(columns={"借方科目": "勘定科目"})
    debit_df["金額"] = debit_df["金額"]
    credit_df = df[["貸方科目", "金額"]].rename(columns={"貸方科目": "勘定科目"})
    credit_df["金額"] = credit_df["金額"] * -1
    combined = pd.concat([debit_df, credit_df])
    grouped = combined.groupby("勘定科目")["金額"].sum().reset_index()
    grouped["区分"] = grouped["勘定科目"].map(account_type_map).fillna("その他")
    bs = grouped.groupby("区分")["金額"].sum().reset_index()
    return grouped, bs

# =========================
# 5. P/L用勘定科目マッピング
# =========================
pl_account_map = {
    # 収益
    "売上高": "収益",
    "受取利息": "収益",
    "雑収入": "収益",
    # 費用
    "仕入高": "費用",
    "旅費交通費": "費用",
    "通信費": "費用",
    "消耗品費": "費用",
    "水道光熱費": "費用",
    "交際費": "費用",
    "支払利息": "費用",
    "租税公課": "費用",
    "減価償却費": "費用",
    "雑費": "費用"
}

def create_profit_loss(df):
    debit_df = df[["借方科目", "金額"]].rename(columns={"借方科目": "勘定科目"})
    debit_df["金額"] = debit_df["金額"] * -1  # 費用扱い
    credit_df = df[["貸方科目", "金額"]].rename(columns={"貸方科目": "勘定科目"})
    credit_df["金額"] = credit_df["金額"]      # 収益扱い
    combined = pd.concat([debit_df, credit_df])
    combined["区分"] = combined["勘定科目"].map(pl_account_map).fillna("その他")
    grouped = combined.groupby("区分")["金額"].sum().reset_index()
    grouped["金額"] = grouped["金額"].round(0)
    # 当期純利益
    profit = grouped.loc[grouped["区分"] == "収益", "金額"].sum() + \
             grouped.loc[grouped["区分"] == "費用", "金額"].sum()
    profit_row = pd.DataFrame([{"区分": "当期純利益", "金額": profit}])
    grouped = pd.concat([grouped, profit_row], ignore_index=True)
    return grouped

# =========================
# 6. Excel出力
# =========================
def export_to_excel(result_df, bs_table, pl_table, file_name="会計レポート.xlsx"):
    with pd.ExcelWriter(file_name, engine="openpyxl") as writer:
        result_df.to_excel(writer, index=False, sheet_name="仕訳一覧")
        bs_table.to_excel(writer, index=False, sheet_name="貸借対照表(BS)")
        pl_table.to_excel(writer, index=False, sheet_name="損益計算書(PL)")
    print(f"✅ Excelファイル出力完了：{file_name}")

# =========================
# 7. サンプルデータ実行
# =========================
data = {
    "日付": ["2025-10-01", "2025-10-02", "2025-10-03"],
    "区分": ["交通費", "備品購入", "売上"],
    "支払方法": ["現金", "銀行振込", "掛け"],
    "摘要": ["電車代（出張）", "デスク購入", "取引先A"],
    "金額": [500, 10000, 50000]
}

df = pd.DataFrame(data)
result_df = process_dataframe(df)
grouped_accounts, bs_table = create_balance_sheet(result_df)
pl_table = create_profit_loss(result_df)
export_to_excel(result_df, bs_table, pl_table, "会計レポート.xlsx")


✅ Excelファイル出力完了：会計レポート.xlsx
